In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO

from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

C:\Users\anil_\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
def run_inference_for_single_image(model, image):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  #image_np = np.array(Image.open(image_path))
  image = np.asarray(Image.open(image))
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)/255
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)
  return (output_dict['dense_1'].numpy()*100).astype(np.int64)

##Option 1 - Reading as a Tensor Flow model

In [9]:
TEST_IMAGE_PATHS = '../input/fruits-360_dataset/fruits-360/Test/Kiwi/r_64_100.jpg'
TEST_IMAGE_PATHS = '../input/fruits-360_dataset/fruits-360/Test/Mango/0_100.jpg'

detection_model = tf.saved_model.load('saved_model/my_model')
fruit_names=["Apple","Avocado","Banana","Cherry","Cocos","Kiwi","Lemon","Mango","Orange"]


predict = run_inference_for_single_image(detection_model, TEST_IMAGE_PATHS)

for index1, cell in enumerate(predict):
    for index, value in enumerate(fruit_names):
        print("In Image", index1,"-", cell[index], "% of", value)
    

In Image 0 - 0 % of Apple
In Image 0 - 0 % of Avocado
In Image 0 - 0 % of Banana
In Image 0 - 0 % of Cherry
In Image 0 - 0 % of Cocos
In Image 0 - 0 % of Kiwi
In Image 0 - 0 % of Lemon
In Image 0 - 99 % of Mango
In Image 0 - 0 % of Orange


In [ ]:
##Option 2 - Reading into CV2

In [12]:
import cv2

TEST_IMAGE_PATH1 = '../input/fruits-360_dataset/fruits-360/Test/Kiwi/r_64_100.jpg'
TEST_IMAGE_PATH2 = '../input/fruits-360_dataset/fruits-360/Test/Mango/0_100.jpg'
TEST_IMAGE_PATH3 = '../input/fruits-360_dataset/fruits-360/Test/Avocado/4_100.jpg'
# Load a model imported from Tensorflow
tensorflowNet = cv2.dnn.readNetFromTensorflow('simple_frozen_graph.pb','tf_label_map.pbtxt')
#tensorflowNet = cv2.dnn.readNetFromTensorflow('saved_model.pb','tf_label_map.pbtxt')
#tensorflowNet = cv2.dnn.readNetFromTensorflow('simple_frozen_graph.pb','tf_label_map.pbtxt')
#tensorflowNet = cv2.dnn.readNetFromTensorflow('simple_frozen_graph.pb','tf_label_map.pbtxt')

# Input image
img = cv2.imread(TEST_IMAGE_PATH1)
rows, cols, channels = img.shape
 
# Use the given image as input, which needs to be blob(s).
tensorflowNet.setInput(cv2.dnn.blobFromImage(img, size=(100, 100), swapRB=True, crop=False))
 
# Runs a forward pass to compute the net output
networkOutput = tensorflowNet.forward()

print(networkOutput)
 
# Loop on the outputs
for detection in networkOutput[0,0]:
    
    score = float(detection[2])
    if score > 0.2:
     
        left = detection[3] * cols
        top = detection[4] * rows
        right = detection[5] * cols
        bottom = detection[6] * rows
 
        #draw a red rectangle around detected objects
        cv2.rectangle(img, (int(left), int(top)), (int(right), int(bottom)), (0, 0, 255), thickness=2)
 
# Show the image with a rectagle surrounding the detected objects 
cv2.imshow('Image', img)
cv2.waitKey()
cv2.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\net.cpp:79: error: (-215:Assertion failed) !empty() in function 'cv::dnn::dnn4_v20220524::Net::forward'
